In [2]:
# -*- coding: utf-8 -*-
"""
Created on Thu Mar 30 13:54:32 2017

@author: vicentej
"""

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
import os
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from P5_functions import *
from P4_functions import *
from sklearn.model_selection import train_test_split
from collections import deque
import pickle
from sklearn.model_selection import GridSearchCV

In [3]:
writeImagesOut = True
basedir = "img/vehicles/"
sub_folders = os.listdir(basedir)
cars = []
for sub_folder in sub_folders:
    print("Car Images > " + basedir + sub_folder + "/*")
    cars.extend(glob.glob(basedir + sub_folder + "/*"))

basedir = "img/non-vehicles/"
sub_folders = os.listdir(basedir)
notcars = []
for sub_folder in sub_folders:
    print("Not Car Images > " + basedir + sub_folder + "/*")
    notcars.extend(glob.glob(basedir + sub_folder + "/*"))

print('Total car images: ' + str(len(cars)))
print('Total not car images: ' + str(len(notcars)))

if (writeImagesOut):
    with open("car_images.txt",'w') as f:
        for fn in cars:
            f.write(fn+'\n')
    with open("not_cars_images.txt",'w') as f:
        for fn in notcars:
            f.write(fn+'\n')
    



Car Images > img/vehicles/GTI_Far/*
Car Images > img/vehicles/GTI_Left/*
Car Images > img/vehicles/GTI_MiddleClose/*
Car Images > img/vehicles/GTI_Right/*
Car Images > img/vehicles/KITTI_extracted/*
Not Car Images > img/non-vehicles/Extras/*
Not Car Images > img/non-vehicles/GTI/*
Total car images: 8792
Total not car images: 8968


In [4]:
### Function for training the model using SVM based on parameters received

def train_model(color_space, orient, pix_per_cell, cell_per_block, hog_channel, spatial_size,
                hist_bins, spatial_feat, hist_feat, hog_feat, x_start_stop, y_start_stop):
    car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
    notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
    
    X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
    # Fit a per-column scaler
    X_scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)

    # Define the labels vector
    y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(
        scaled_X, y, test_size=0.2, random_state=rand_state)
    
    #svc = LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
    #     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
    #     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
    #     verbose=1)
    svc = LinearSVC()
    svc.fit(X_train, y_train)
    score = svc.score(X_test, y_test)
    return svc, score, X_scaler


def train_model_optimized(color_space, orient, pix_per_cell, cell_per_block, hog_channel, spatial_size,
                hist_bins, spatial_feat, hist_feat, hog_feat, x_start_stop, y_start_stop):
    car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
    notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
    
    X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
    # Fit a per-column scaler
    X_scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)

    # Define the labels vector
    y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

    # Split up data into randomized training and test sets
    
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(
        scaled_X, y, test_size=0.2, random_state=rand_state)
    
    #svc = LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
    #     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
    #     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
    #     verbose=1)
    
    parameters = {'kernel':('linear', 'rbf'), 'C':[0.01, 0.1, 1, 10],'gamma':[0.01, 0.1, 1, 10]}
    svr = SVC()
    clf = GridSearchCV(svr, parameters)
    clf.fit(X_train, y_train)
    print(clf.best_params_)
   
    score = svr.score(X_test, y_test)
    return svr, score, X_scaler


In [5]:
### Parameters configuration

color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 10  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (32, 32) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
#x_start_stop = [600, 1280] # Min and max in y to search in slide_window()
x_start_stop = [None, None]
y_start_stop = [400, 656] # Min and max in y to search in slide_window()

In [6]:
### Obtain a model (either retraining or from previous training)

retrain = False
find_optimum = False

if (retrain):
    if (find_optimum):
        svc, score, X_scaler = train_model_optimized(color_space, orient, pix_per_cell, cell_per_block, hog_channel, spatial_size,
                      hist_bins, spatial_feat, hist_feat, hog_feat, x_start_stop, y_start_stop)
    else:
        svc, score, X_scaler = train_model(color_space, orient, pix_per_cell, cell_per_block, hog_channel, spatial_size,
                      hist_bins, spatial_feat, hist_feat, hog_feat, x_start_stop, y_start_stop)
    model_data = [svc, score, X_scaler]
    pickle.dump(model_data, open( "P5_model.p", "wb" ))
else:
    model_data = pickle.load(open( "P5_model.p", "rb" ))
    svc, score, X_scaler = model_data[0], model_data[1], model_data[2]


# Check the score of the SVC
print('Test accuracy: ', round(score, 4))


Test accuracy:  0.9893


In [7]:
### Functions for processing images
from PIL import Image

def process_image_persistent(image):    
    #out_img = locate_cars(image, heat_list)
    t_hl = heat_list
    out_img, t_hl = locate_cars_persistent(image, heat_list, svc, X_scaler, y_start_stop, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, heat_thresold, True, False)
    global heat_list
    heat_list = t_hl
    return out_img

def process_image_simple(image):    
    out_img = locate_cars(image, svc, X_scaler, y_start_stop, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    return out_img

def process_image_full(image):
    t_hl = heat_list
    out_img, t_hl = locate_cars_persistent(image, heat_list, svc, X_scaler, y_start_stop, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, heat_thresold, True, False)
    global heat_list
    heat_list = t_hl
    lane_image = locate_lane(out_img, last_data, mtx, dist)
    return lane_image 

<ipython-input-7-97a90c45454c>:8: SyntaxWarning: name 'heat_list' is used prior to global declaration
  global heat_list
<ipython-input-7-97a90c45454c>:19: SyntaxWarning: name 'heat_list' is used prior to global declaration
  global heat_list


In [8]:
### Video processing
from moviepy.editor import VideoFileClip
from IPython.display import HTML

heat_list = deque(maxlen = 8)
heat_list.clear()
heat_thresold = 6

last_data = []
objpoints, imgpoints = calibrate()
# Do camera calibration given object points and image points
img = cv2.imread('camera_cal/test_image.jpg')
img_size = (img.shape[1], img.shape[0])
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)

video_out = 'project_video_out_final.mp4'
clip1 = VideoFileClip("project_video.mp4")
#clip1 = VideoFileClip("out_22_25.mp4")
#clip1 = VideoFileClip("challenge_video_shorter.mp4")
challenge_clip = clip1.fl_image(process_image_full) #NOTE: this function expects color images!!
%time challenge_clip.write_videofile(video_out, audio=False)


[MoviePy] >>>> Building video project_video_out_final.mp4
[MoviePy] Writing video project_video_out_final.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [09:05<00:00,  2.31it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_out_final.mp4 

Wall time: 9min 6s


In [9]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_out))